# Running a TRUST computation for big cases

### Description: 

This form explains what needs to be done if you want to perform a TRUST simulation with a number of entities in your mesh (number of nodes, or number of elements) exceeding the 32b integer limits (typically above 1Bio entities).

In TRUST, a typical data set looks like this:

```c
dimension 2
Pb_Thermohydraulique pb

Domaine dom

Read_MED { domaine dom file ./MESH/geom.med }
/* or :
    Mailler dom {
     Pave cavit {
     ...
    }
} */

/* Splitting instruction - only used when creating the sub-domains for parallel computation : */
Partition dom
{
    Partition_tool Tranche { tranches 2 1 }
    ...
}

/* Scatter instruction - only used when executing the test case in parallel : 
Scatter DOM1.Zones dom_solide 
*/

Scheme_euler_explicit sch
Read sch
{
 ...
}
...

```


The dataset above can be executed both in sequential and in parallel.

## Transforming the partionning part for 64b

If you want to go above the 32b limit, **only the dataset used for partitionning needs to be amended**. Below is typically an extract of the above dataset, cut just before the 'Scatter' keyword.

The dataset used for partitionning must now use the '_64' objects:

```c
dimension 2
Pb_Thermohydraulique pb

Domaine_64 dom

Read_MED_64 { domaine dom file ./MESH/geom.med }
/* or :
    Mailler_64 dom {
     Pave_64 cavit {
     ...
    }
} */

/* Splitting instruction - only used when creating the sub-domains for parallel computation : */
Partition_64 dom
{
    Partition_tool Tranche_64 { tranches 2 1 }
    ...
}

```

Notice how the following objects have been changed:
- 'domaine_64'
- 'read_med_64' (or 'Mailler_64' and 'Pave_64')
- 'partition_64'
- 'tranche_64' (the partitionning tool)

These 64b version of the objects can only be used for partioning (and early geometric transformations). Notably you can **not** perform a computation with it (see below), and hence a classical sequential (non parallel) computation can not be done with them.

## Transforming the parallel dataset for execution

The part of the dataset used for the parallel run itself remains unchanged compared to a standard TRUST computation, and looks like this:

```c
dimension 2
Pb_Thermohydraulique pb

Domaine dom

Scatter DOM1.Zones dom_solide 

Scheme_euler_explicit sch
Read sch
{
 ...
}
...
```

Notice how we use a simple 'Domaine' again (not a 'Domaine_64') since this dataset now loads only part of the big domain on each proc, and this always smaller, and will fit within 32b.

## Test case
In the TRUST test database, the test case "MED_docond_64" performs a 64b computation. The syntax of the dataset is a bit akward just so that it passes the automatic TRUST checks in both sequential and parallel.